In [1]:
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from main import init_config
from controller import Controller
from dataset.process import TokenParser

import numpy as np

ctrl = Controller(*init_config())

2022-01-09 18:09:04.494688: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Running the DreamProlog algorithm.
There was a problem with the provided arguments. The program will run in the default setting:
--configs prolog --logdir logdir
Namespace(act='elu', action_gnn_hidden_act=64, action_gnn_hidden_val=200, action_gnn_layers=6, action_gnn_next_shape=(32, 64, 32), action_gnn_start_shape=(4, 1, 4), action_repeat=1, actor_disc=5, actor_dist='normal', actor_entropy='1e-4', actor_grad_clip=100, actor_init_std=1.0, actor_layers=4, actor_lr=0.001, actor_min_std=0.1, actor_outscale=0.0, actor_state_entropy=0.0, actor_temp=0.1, atari_grayscale=False, behavior_stop_grad=True, clip_rewards='identity', collect_dyn_sample=True, dataset_size=100000, debug=False, disag_layers=1, disag_log=True, disag_models=3, disag_offset=0, disag_target='embed', disag_units=200, discount=0.95, discount_lambda=0.8, discount_layers=2, discount_scale=1.0, dyn_cell='gru', dyn_deter=512, dyn_discrete=32, dyn_hidden=512, dyn_input_layers=5, dyn_mean_act='none', dyn_min_std=0.1, dyn_output_lay

usage: ipykernel_launcher.py [-h] --configs CONFIGS [CONFIGS ...] --logdir
                             LOGDIR
ipykernel_launcher.py: error: the following arguments are required: --configs, --logdir


Logdir logdir


2022-01-09 18:09:05.901674: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-09 18:09:05.902345: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-09 18:09:07.217588: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-01-09 18:09:07.217620: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-09 18:09:07.219046: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-09 18:09:07.219077: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-01-09 18

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/l18_zfmisc_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t40_funct_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Prefill dataset (0 steps).
Loaded problem: m2n140__t58_partfun1
Eval episode has 7 steps and return -0.200.


/home/kristsz/DreamProLog/envs/wrappers.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  value = np.array(value)


In [2]:
meta = ctrl.datasetManager._train_eps._meta
ds = []
for v in meta.values():
    ds.extend([str(s)[1:] for s in v['action_space_text']])
    
parser = TokenParser()

def pad(narr):
    size = narr.size
    return np.pad(narr, [1, 128-size], constant_values= [(299, 0)])

parsed_ds = []
for dp in ds:
    parsed_ds.append(pad(np.array(parser.parse(dp), dtype = np.int32)))

In [3]:
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    for i in range(1000):
        x = tf.constant([parsed_ds[np.random.randint(len(parsed_ds))] for i in range(BATCH_SIZE)])
        yield x[:, :-1], x[:, 1:]
sgn = tf.TensorSpec((BATCH_SIZE, 128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))

2022-01-09 18:09:18.289218: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-09 18:09:18.292047: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-01-09 18:09:18.292101: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-09 18:09:18.292144: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-09 18:09:18.292171: I tensorflow/stream_executor/platform/default/ds

In [4]:
from unorganized.parts5 import Net
from unorganized.loss import loss_function


GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
num_layers = 3
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

model = Net(num_layers, 16, 128, d_model, num_heads, dff, dropout_rate)

In [6]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.0):
        super().__init__()
        self.omega = omega
        self.loss_objective = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        
    def call(self, y_true, y_pred):
        loss = self.loss_objective(y_true, y_pred)
        mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
        
        loss *= mask
        return loss
    
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=6000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    
def accuracy_function(real, pred):
    accuracies = tf.equal(real, tf.cast(tf.math.argmax(pred, axis=-1), dtype=tf.int32))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [7]:
loss = Loss()
x, x = next(iter(tf_ds))
y_pred = model(x, False)
loss(x, y_pred)
accuracy_function(x, y_pred)

2022-01-09 18:09:19.105122: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-09 18:09:19.125599: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3598190000 Hz
2022-01-09 18:09:19.157462: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-01-09 18:09:19.875184: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [8]:
learning_rate = CustomSchedule(d_model)
optimizer=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(optimizer = optimizer, loss=Loss(), metrics=[])


In [9]:
model.fit(tf_ds, epochs = 50)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (

KeyboardInterrupt: 

In [14]:
#To visualize performance
x, y_true = next(iter(tf_ds))
_y_pred = model(x, False)
_y_pred = tf.nn.softmax(_y_pred, axis=-1)
y_probs = tf.gather(_y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(_y_pred, axis=-1)
print(x[:4], y_pred[:4], tf.cast(100*y_probs[:4], dtype=tf.int32), tf.cast(100*tf.reduce_max(_y_pred[:4], axis = -1), dtype = tf.int32), sep='\n\n')

tf.Tensor(
[[299   3 158  23  12   2  37  23  12   2   3  33  23  12   2 157  23  12
    2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [299   9   5  12  13   2   9   5  16  49  23  13   2   3  14   5  12  16
    2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   

In [ ]:
mask = tf.cast(tf.math.equal(x[:], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
        
v = model.enc_embed(x)[:]#, model.encoder.variable#tf.tile(model.encoder.variable, (32, 1, 1))
vs = [v]
att = []
for layer in model.encoder.layers:
    _v, a = mha(v, v, v, mask)
    v = v + _v
    v = sl(v) 
    vs.append(v)
    att.append(a)

In [11]:
mask = tf.cast(tf.math.equal(x[:], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
v = model.enc_embed(x)[:]
embed, att = model.encoder(v, mask, False)

In [13]:
embed[:, :1, :8]

<tf.Tensor: shape=(32, 1, 8), dtype=float32, numpy=
array([[[ 0.0361836 , -0.10256108,  0.4023145 , -0.24285649,
         -0.17512053,  0.23289916, -0.36779302,  0.44178143]],

       [[-0.31037804,  0.32174262, -0.18416238, -0.3617726 ,
          0.5733119 ,  0.01409043, -0.10353428, -0.22446443]],

       [[ 0.00805724, -0.0114899 ,  0.15925747, -0.29105526,
          0.31619775, -0.17798011,  0.2296477 , -0.01207742]],

       [[-0.05382052,  0.10841717,  0.692728  ,  0.11222706,
         -0.21121293,  0.8914639 ,  0.03716245,  0.18558213]],

       [[-0.13097747,  0.7052641 , -0.7361518 , -0.15868004,
          0.10542033, -0.15197513, -0.17474407,  0.65583575]],

       [[-0.4998785 ,  0.6584555 ,  0.27690056,  0.26485413,
          0.03289813, -0.0220038 , -0.19893831,  0.0268001 ]],

       [[-0.03642412, -0.06805982, -0.3231477 , -0.03467386,
          0.10892622,  0.85996616,  0.7717775 ,  0.54800034]],

       [[-0.40738094,  0.58111465,  0.7478768 ,  0.15178776,
          0.

In [ ]:
tf.reduce_sum(tf.cast(att[0][1]>0.01, tf.int32))/(2**(5+3+7+4)) #32, 8, 128, 16

In [ ]:
np.prod(att[0][0].shape)

In [ ]:
import matplotlib.pyplot as plt

def analyze_att(matrix):
    print(matrix.shape)
    entries = np.prod(matrix.shape)
    x, y = [], []
    for i in range(2, 100):
        y.append(tf.reduce_sum(tf.cast(matrix>1/i, tf.float32)/entries))
        x.append(i)
    plt.plot(x, y)
    plt.show()

In [ ]:
analyze_att(att[2][1])

In [ ]:
tf.math.argmax(embed, axis=-1), tf.reduce_max(embed, axis=-1)

In [ ]:
model.encoder.variable


In [ ]:
vs

In [ ]:
pred = model.dense(vs[-1])

#### 

In [ ]:
tf.math.argmax(pred[1, : , :], axis= -1 ), x[1]

In [ ]:
tf.math.reduce_max(pred[1], axis=-1)